# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [124]:
import csv
import json
import os
import requests
import this
from typing import final

import pandas as pd
from pandas.core.interchange.dataframe_protocol import DataFrame

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [141]:
def list_directories(path):
    # List only directories
    directories = [os.path.join(path, d) for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    return directories

def list_files(directory):
    # List only files
    files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    return files

def read_json(path: str):
    with open(path) as json_file:
        json_data = json.load(json_file)
        project_artifact = json_data["projectArtifact"]
        file_import_reports = json_data["fileImportReports"]
        project_unused_dependencies = json_data["projectUnusedDependencies"]
        project_use_dependencies = json_data["projectUseDependencies"]
        return project_artifact, file_import_reports, project_use_dependencies, project_unused_dependencies

def list_json_files(directory):
    try:
        return [f for f in os.listdir(directory) if f.endswith('.json')]
    except FileNotFoundError:
        print(f"Error: Directory '{directory}' not found.")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []
    
def calculate_import_but_unused_dependencies(project_unused_dependencies, file_import_reports):
    project_unused_dependencies_list = []
    result = []
    for unused_dependency in project_unused_dependencies:
        artifact = unused_dependency["artifact"]
        project_unused_dependencies_list.append(artifact)
    
    for file in file_import_reports:
        unused_import_report = file["unusedImportReport"]
        for unused_import in unused_import_report:
            artifact = unused_import["artifact"]["artifact"]
            if (artifact in project_unused_dependencies_list):
                result.append(artifact)
    return len(set(result))

def calculate_file_level_unused_import(file_import_reports):
    total_use_import = 0
    total_unused_import = 0
    num_files = 0
    for file_report in file_import_reports:
        num_use_import_report = len(file_report["useImportReport"])
        num_unused_import_report = len(file_report["unusedImportReport"])
        total_use_import += num_use_import_report
        total_unused_import += num_unused_import_report
        num_files += 1
    return total_use_import, total_unused_import, num_files
    
# read_json("data/new-dependency-output/at.molindo.social:spring-social-openid:1.1.1.RELEASE.json")

In [79]:
class DependenciesDetails:
    group_df = None
    df = None
    
    def __init__(self):
        csv_file = "data/artifact-dependency-details.csv"
        self.df = pd.read_csv(csv_file)
        
    def get_artifact_details(self, release_id):
        # Group by 'release_id' and get the first row of each group
        return self.df[self.df['release_id'] == release_id]

        

In [145]:
dir_list = list_directories("/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-3")
json_file_list = []
print(dir_list)
for dir in dir_list:
    files = list_files(dir)
    print(files)
    json_file_list.extend(files)
dependency_details = DependenciesDetails()

list_df = []
for file in json_file_list:
    project_artifact, file_import_reports, project_use_dependencies, project_unused_dependencies = read_json(file)
    row = dependency_details.get_artifact_details(project_artifact).copy()
    import_but_unused = calculate_import_but_unused_dependencies(project_unused_dependencies, file_import_reports)
    num_using_import_in_file, num_unused_import_in_file, num_files = calculate_file_level_unused_import(file_import_reports)
    row['num_use_import'] = len(project_use_dependencies)
    row['num_unused_import'] = len(project_unused_dependencies)
    row['num_import_but_unused'] = import_but_unused
    row['num_use_import_in_file'] = num_using_import_in_file
    row['num_unused_import_in_file'] = num_unused_import_in_file
    row['avg_use_import_in_each_file'] = num_using_import_in_file / num_files
    row['avg_unused_import_in_each_file'] = num_unused_import_in_file / num_files
    row['num_files'] = num_files
    list_df.append(row)
    
final_df = pd.concat(list_df, ignore_index=True)
final_df

['/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-3/nl.hsac:hsac-fitnesse-email', '/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-3/io.github.stefanofornari:vfs2nio', '/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-3/com.squareup.pagerduty:pagerduty-incidents', '/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-3/uk.co.rgordon:oj-ssh', '/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-3/de.vandermeer:ascii-utf-themes', '/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-3/io.quarkus.qlue:qlue', '/Users/nabhansuwanachote/Desktop/research/msr-2025-challenge/java-dependency-analyzer/new-dependency-output-3/com.forgerock.openbanking:openban

,artifact_id,release_id,version,timestamp,popularity,all_dependency,compile_dependency,provided_dependency,dependency,num_use_import,num_unused_import,num_import_but_unused,num_use_import_in_file,num_unused_import_in_file,avg_use_import_in_each_file,avg_unused_import_in_each_file,num_files
0,nl.hsac:hsac-fitnesse-email,nl.hsac:hsac-fitnesse-email:0.5.0,0.5.0,1574153748000,0,4,2,1,"[""nl.hsac:hsac-fitnesse-fixtures:4.6.3:compile...",2,1,0,42,0,14.000000,0.000000,3
1,io.github.stefanofornari:vfs2nio,io.github.stefanofornari:vfs2nio:0.11.6,0.11.6,1709454582000,0,9,1,0,"[""org.assertj:assertj-core:3.24.2:test"",""junit...",1,8,0,198,0,13.200000,0.000000,15
2,com.squareup.pagerduty:pagerduty-incidents,com.squareup.pagerduty:pagerduty-incidents:2.0.0,2.0.0,1458181265000,2,7,3,0,"[""com.squareup.retrofit2:retrofit-mock:2.0.0:t...",2,5,0,16,7,1.777778,0.777778,9
3,uk.co.rgordon:oj-ssh,uk.co.rgordon:oj-ssh:1.6.0,1.6.0,1674342182000,0,12,4,1,"[""org.apache.sshd:sshd-putty:2.5.1:compile"",""o...",4,7,0,109,0,9.909091,0.000000,11
4,de.vandermeer:ascii-utf-themes,de.vandermeer:ascii-utf-themes:0.0.1,0.0.1,1491317289000,10,3,1,0,"[""junit:junit:4.12:test"",""de.sven-jacobs:lorem...",1,2,0,124,0,2.101695,0.000000,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,com.github.tcl-ec:tclsms,com.github.tcl-ec:tclsms:1.1.0,1.1.0,1565172555000,0,5,4,0,"[""org.json:json:20170516:compile"",""org.apache....",3,1,0,78,0,4.333333,0.000000,18
440,com.apitrary:apitrary-transport-httpclient,com.apitrary:apitrary-transport-httpclient:1.0.0,1.0.0,1371156534000,0,6,5,0,"[""commons-io:commons-io:2.4:compile"",""log4j:lo...",4,2,0,19,0,9.500000,0.000000,2
441,io.github.nikosrig:mastercard-ais,io.github.nikosrig:mastercard-ais:1.1.0,1.1.0,1669752984000,0,5,3,0,"[""com.mastercard.developer:oauth1-signer:1.5.2...",3,2,0,141,0,2.238095,0.000000,63
442,dev.mccue:guava,dev.mccue:guava:33.2.0,33.2.0,1716314081000,0,13,13,0,"[""dev.mccue:guava-base:33.2.0:compile"",""dev.mc...",0,13,0,0,0,0.000000,0.000000,1


In [150]:
summary_df = final_df[['num_use_import', 'num_unused_import', 'num_import_but_unused', 'avg_use_import_in_each_file', 'avg_unused_import_in_each_file', 'num_files']]

summary = pd.DataFrame({
    'mean': summary_df.mean(),
    'median': summary_df.median(),
    'max': summary_df.max(),
    'min': summary_df.min(),
    'std': summary_df.std()  # Standard deviation
})

# Rename index to show column names from the original DataFrame
summary.index.name = 'Column'

# Display the summary table
summary

,mean,median,max,min,std
Column,,,,,
num_use_import,3.324324,3.0,48.0,0.0,3.706896
num_unused_import,3.274775,2.0,22.0,0.0,3.173848
num_import_but_unused,0.009009,0.0,1.0,0.0,0.094594
avg_use_import_in_each_file,7.033053,5.7,45.0,0.0,5.179182
avg_unused_import_in_each_file,0.083450,0.0,5.0,0.0,0.360627
num_files,37.650901,12.0,744.0,1.0,84.160264


In [148]:
summary

,mean,median,max,min,std
Column,,,,,
num_use_import,3.324324,3.0,48.0,0.0,3.706896
num_unused_import,3.274775,2.0,22.0,0.0,3.173848
num_import_but_unused,0.009009,0.0,1.0,0.0,0.094594
avg_use_import_in_each_file,7.033053,5.7,45.0,0.0,5.179182
avg_unused_import_in_each_file,0.083450,0.0,5.0,0.0,0.360627
num_files,37.650901,12.0,744.0,1.0,84.160264
